In [5]:
import torch
from torch import nn
import torch.nn.functional as F

In [6]:
import pandas as pd

test_df = pd.read_csv("/kaggle/input/mnist-cv-test/mnist_test.csv")

train_df = pd.read_csv("/kaggle/input/mnist-cv-test/mnist_train.csv")



In [7]:
# Function to plot sample images
from tqdm import tqdm

def to_image(df):

    new_df = pd.DataFrame()
    images = []
    labels = []
    for i in tqdm(range(len(df))):
        # Reshape the flattened image to 28x28
        
        image = df.iloc[i, 1:].values.reshape(28, 28) #resize
        image = image/255.0 # Normalize
        
        label = df.iloc[i, 0]
        
        images.append(image)
        labels.append(label)
        
    new_df['images'] = images
    new_df['labels'] = labels

    return new_df
# Plot sample images from the training data
new_train_df = to_image(train_df)
new_test_df = to_image(test_df)

# Plot sample images from the testing data

# new_test_path = "/kaggle/working/test_images.csv"
# new_train_path = "/kaggle/working/train_images.csv"

# new_train_df.to_csv(new_train_path)
# new_test_df.to_csv(new_test_path)

# print(f"images saved to {new_train_path} and {new_test_path}")


100%|██████████| 9999/9999 [00:01<00:00, 9346.22it/s]


In [13]:
## dataset split
X_train = new_train_df['images'].values
y_train = new_train_df['labels']

test_images = new_test_df['images'].values
test_labels = new_test_df['labels']

y_train = torch.tensor(y_train, dtype=torch.long)
test_labels = torch.tensor(test_labels, dtype=torch.long)

# One-hot encode the labels
num_classes = 10  # Number of classes (0 to 9)
y_train = torch.nn.functional.one_hot(y_train, num_classes=num_classes)
test_labels = torch.nn.functional.one_hot(test_labels, num_classes=num_classes)


X_test = test_images[:len(test_images)//2]
X_val = test_images[len(test_images)//2:]

y_test = test_labels[:len(test_labels)//2]
y_val = test_labels[len(test_labels)//2:]



len(X_train),len(y_train),len(X_test),len(X_val ),len(y_test),len(y_val )

(59999, 59999, 4999, 5000, 4999, 5000)

In [14]:
import numpy as np
for i in tqdm(range(len(X_train))):
    X_train[i] = list(X_train[i])
X_train = list(X_train)
X_train = np.array(X_train, dtype=np.float32)  
X_train = torch.tensor(X_train)

100%|██████████| 59999/59999 [00:00<00:00, 1101369.18it/s]


In [15]:
X_train = X_train.unsqueeze(1)
X_train.shape

torch.Size([59999, 1, 28, 28])

In [16]:
for i in tqdm(range(len(X_test))):
    X_test[i] = list(X_test[i])
X_test = list(X_test)
X_test = np.array(X_test, dtype=np.float32)  
X_test = torch.tensor(X_test)
X_test = X_test.unsqueeze(1)


for i in tqdm(range(len(X_val))):
    X_val[i] = list(X_val[i])
X_val = list(X_val)
X_val = np.array(X_val, dtype=np.float32)  
X_val = torch.tensor(X_val)
X_val = X_val.unsqueeze(1)

100%|██████████| 5000/5000 [00:00<00:00, 1101734.70it/s]


In [17]:
y_train = torch.tensor(y_train)
# y_train = y_train.unsqueeze(1)
y_train.shape

<ipython-input-17-aa5db8dc8cc2>:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.tensor(y_train)


torch.Size([59999, 10])

In [19]:
y_test = torch.tensor(y_test)
# y_test = y_test.unsqueeze(1)

y_val = torch.tensor(y_val) 
# y_val = y_val.unsqueeze(1)

<ipython-input-19-7d64a92bd736>:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_test = torch.tensor(y_test)
<ipython-input-19-7d64a92bd736>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_val = torch.tensor(y_val)


In [20]:
from torch.utils.data import DataLoader, Dataset

class MnistDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

In [21]:

batch_size = 32

train_dataset = MnistDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = MnistDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)



val_dataset = MnistDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


In [22]:
import torch
from torch import nn
import torch.nn.functional as F

In [23]:
import torch
import torch.nn as nn

class Block_1(nn.Module):
    def __init__(self):
        super(Block_1, self).__init__()
        self.relu = nn.ReLU()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.batchnorm1 = nn.BatchNorm2d(64)
        self.maxpool1 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=(0, 0))  # Reduced kernel size and stride

    def forward(self, x):
        x = self.conv1(x)
        x = self.batchnorm1(x)
        x = self.relu(x)
        op1 = self.maxpool1(x)
        return op1


class Block_2_1(nn.Module):
    def __init__(self):
        super(Block_2_1, self).__init__()
        self.relu = nn.ReLU()
        self.dropout_percentage = 0.5

        self.conv2_1_1 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.batchnorm2_1_1 = nn.BatchNorm2d(64)
        self.conv2_1_2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.batchnorm2_1_2 = nn.BatchNorm2d(64)
        self.dropout2_1 = nn.Dropout(p=self.dropout_percentage)

    def forward(self, op1):
        x = self.conv2_1_1(op1)
        x = self.batchnorm2_1_1(x)
        x = self.relu(x)
        x = self.conv2_1_2(x)
        x = self.batchnorm2_1_2(x)
        x = self.dropout2_1(x)
        op2_1 = self.relu(x + op1)
        return op2_1


class Block_2_2(nn.Module):
    def __init__(self):
        super(Block_2_2, self).__init__()
        self.relu = nn.ReLU()
        self.dropout_percentage = 0.5

        self.conv2_2_1 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.batchnorm2_2_1 = nn.BatchNorm2d(64)
        self.conv2_2_2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.batchnorm2_2_2 = nn.BatchNorm2d(64)
        self.dropout2_2 = nn.Dropout(p=self.dropout_percentage)

    def forward(self, op2_1):
        x = self.conv2_2_1(op2_1)
        x = self.batchnorm2_2_1(x)
        x = self.relu(x)
        x = self.conv2_2_2(x)
        x = self.batchnorm2_2_2(x)
        x = self.dropout2_2(x)
        op2 = self.relu(x + op2_1)
        return op2


class Block_3(nn.Module):
    def __init__(self):
        super(Block_3, self).__init__()
        self.relu = nn.ReLU()
        self.dropout_percentage = 0.5

        self.conv3_1_1 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.batchnorm3_1_1 = nn.BatchNorm2d(128)
        self.conv3_1_2 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.batchnorm3_1_2 = nn.BatchNorm2d(128)

        # Skip connection
        self.concat_adjust_3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(1, 1), stride=(2, 2), padding=(0, 0))
        self.dropout3_1 = nn.Dropout(p=self.dropout_percentage)

        self.conv3_2_1 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.batchnorm3_2_1 = nn.BatchNorm2d(128)
        self.conv3_2_2 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.batchnorm3_2_2 = nn.BatchNorm2d(128)
        self.dropout3_2 = nn.Dropout(p=self.dropout_percentage)

    def forward(self, op2):
        x = self.conv3_1_1(op2)
        x = self.batchnorm3_1_1(x)
        x = self.relu(x)
        x = self.conv3_1_2(x)
        x = self.batchnorm3_1_2(x)
        x = self.dropout3_1(x)

        op2 = self.concat_adjust_3(op2)  # Skip connection
        op3_1 = self.relu(x + op2)

        x = self.conv3_2_1(op3_1)
        x = self.batchnorm3_2_1(x)
        x = self.relu(x)
        x = self.conv3_2_2(x)
        x = self.batchnorm3_2_2(x)
        x = self.dropout3_2(x)
        op3 = self.relu(x + op3_1)

        return op3


class Resnet18(nn.Module):
    def __init__(self, n_classes):
        super(Resnet18, self).__init__()

        self.block1 = Block_1()
        self.block2_1 = Block_2_1()
        self.block2_2 = Block_2_2()
        self.block3 = Block_3()

        # Final block
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))  # Dynamically adjust pooling size
        self.fc = nn.Linear(in_features=128, out_features=1000)
        self.out = nn.Linear(in_features=1000, out_features=n_classes)

    def forward(self, x):
        op1 = self.block1(x)
        op2_1 = self.block2_1(op1)
        op2 = self.block2_2(op2_1)
        op3 = self.block3(op2)

        x = self.avgpool(op3)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)
        x = self.out(x)

        return x

In [61]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

import torch.optim as optim



class EarlyStopping:
    def __init__(self, patience=5, min_delta=0):
        
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = float('inf')
        self.early_stop = False

    def __call__(self, val_loss):
        if val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True


output_size = 10
learning_rate = 0.001
batch_size = 32
num_epochs = 2 #10
n_classes = 10
# Instantiate the model, loss function, and optimizer
model = Resnet18(n_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)
early_stopping = EarlyStopping(patience=5, min_delta=0.01)




In [54]:



def calculate_accuracy(loader, model):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in tqdm(loader , desc = "Processing accuracy..." ):
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)  # Get predicted class indices

            # If labels are one-hot encoded, convert them to class indices
            if labels.dim() > 1 and labels.size(1) > 1:  # Check if labels are one-hot encoded
                labels = torch.argmax(labels, dim=1)  # Convert to class indices

            # Compare predicted and true class indices
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

            # print(correct / total)

    
    return 100 * correct / total




In [62]:
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0

    for i, (inputs, labels) in tqdm(enumerate(train_loader) , desc = "training over Batches"):
        
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        outputs = outputs.squeeze(1)

        # labels = labels.long()
        labels = labels.float()

        try:
        # print(outputs.shape,labels.shape)
            loss = criterion(outputs, labels)
        except Exception as e:
            print(e)
            continue
        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

        if (i + 1) % 100 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {loss.item():.4f}')


    
    
    model.eval()
    test_loss = 0.0
    with torch.no_grad():
        for inputs, labels in tqdm(test_loader , desc = "calculating test loss over an epoch.."):
            outputs = model(inputs)
            outputs = outputs.squeeze(1)


            labels = labels.float()
            loss = criterion(outputs, labels)
            test_loss += loss.item()
    
    # Calculate Loss

    
    
    train_loss /= len(train_loader)
    test_loss /= len(test_loader)


    
    # Calculate accuracy
    train_acc = calculate_accuracy(train_loader, model)
    test_acc = calculate_accuracy(test_loader, model)


    

    print(f'Epoch [{epoch + 1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {test_loss:.4f} '
          f'Train Accuracy: {train_acc:.2f}%, Test Accuracy: {test_acc:.2f}%')


    scheduler.step(test_loss)

    early_stopping(test_loss)
    if early_stopping.early_stop:
        print("Early stopping triggered!")
        break
print('Training complete.')

training over Batches: 102it [00:09, 10.73it/s]

Epoch [1/2], Step [100/1875], Loss: 0.2287


training over Batches: 202it [00:18, 10.79it/s]

Epoch [1/2], Step [200/1875], Loss: 0.0714


training over Batches: 302it [00:29,  9.82it/s]

...

Epoch [2/2], Step [1700/1875], Loss: 0.0613


training over Batches: 1802it [02:53, 10.55it/s]

Epoch [2/2], Step [1800/1875], Loss: 0.0214


training over Batches: 1875it [03:00, 10.40it/s]
Processing accuracy...: 100%|██████████| 157/157 [00:04<00:00, 35.38it/s]

Epoch [2/2], Train Loss: 0.0749, Val Loss: 0.0487 Train Accuracy: 98.83%, Test Accuracy: 98.42%
Training complete.


In [52]:





def calculate_accuracy(loader, model):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in tqdm(loader , desc = "Processing accuracy..." ):
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)  # Get predicted class indices

            # If labels are one-hot encoded, convert them to class indices
            if labels.dim() > 1 and labels.size(1) > 1:  # Check if labels are one-hot encoded
                labels = torch.argmax(labels, dim=1)  # Convert to class indices

            # Compare predicted and true class indices
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

            # print(correct / total)

    
    return 100 * correct / total



train_acc = calculate_accuracy(train_loader, model)

Processing accuracy...: 100%|██████████| 1875/1875 [00:54<00:00, 34.10it/s]


In [51]:
train_acc

98.35163919398657

In [33]:
total

NameError: name 'total' is not defined

In [63]:
import torch
from sklearn.metrics import precision_score, recall_score
import numpy as np

def calculate_metrics(loader, model):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)  # Get predicted class indices
            
            if labels.dim() > 1 and labels.size(1) > 1:  # Check if labels are one-hot encoded
                labels = torch.argmax(labels, dim=1)

            
            all_preds.extend(predicted.cpu().numpy())
            
            all_labels.extend(labels.cpu().numpy())

    # Convert lists to numpy arrays
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)

    # Calculate accuracy
    accuracy = 100 * (all_preds == all_labels).sum() / len(all_labels)

    # Calculate precision and recall
    precision = precision_score(all_labels, all_preds, average='macro', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='macro', zero_division=0)

    return accuracy, precision, recall

accuracy, precision, recall = calculate_metrics(val_loader, model)

# Print results
print(f"Accuracy: {accuracy:.2f}%")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")

Accuracy: 99.30%
Precision: 0.9931
Recall: 0.9929


In [72]:
import torch
import torch.nn as nn
import torch.onnx


# Instantiate the model
model = Resnet18(n_classes = 10)

# Set the model to evaluation mode
model.eval()

dummy_input = train_dataset[1][0].unsqueeze(0)  # Example: batch size of 1, 32 channels, 28x28 image

# Define the output ONNX file path
onnx_file_path = "/kaggle/working/resnet18_v1.onnx"

# Export the model to ONNX
torch.onnx.export(
    model,                  # Model to export
    dummy_input,            # Dummy input to the model
    onnx_file_path,         # Output ONNX file path
    export_params=True,     # Store the trained parameter weights inside the model file
    opset_version=11,       # ONNX opset version to use
    do_constant_folding=True,  # Optimize the model by folding constants
    input_names=['input'],  # Name of the input node
    output_names=['output'],  # Name of the output node
    dynamic_axes={          # Dynamic axes for variable batch size
        'input': {0: 'batch_size'},
        'output': {0: 'batch_size'},
    }
)

print(f"Model exported to {onnx_file_path}")

Model exported to /kaggle/working/resnet18_v1.onnx


In [79]:
import onnx

# Load the ONNX model
onnx_model = onnx.load("/kaggle/working/resnet18_v1.onnx")

# Check that the model is well-formed
onnx.checker.check_model(onnx_model)

# Print a human-readable representation of the model
print(onnx.helper.printable_graph(onnx_model.graph))

graph main_graph (
  %input[FLOAT, batch_sizex1x28x28]
) initializers (
  %block3.concat_adjust_3.weight[FLOAT, 128x64x1x1]
  %block3.concat_adjust_3.bias[FLOAT, 128]
  %fc.weight[FLOAT, 1000x128]
  %fc.bias[FLOAT, 1000]
  %out.weight[FLOAT, 10x1000]
  %out.bias[FLOAT, 10]
  %onnx::Conv_115[FLOAT, 64x1x3x3]
  %onnx::Conv_116[FLOAT, 64]
  %onnx::Conv_118[FLOAT, 64x64x3x3]
  %onnx::Conv_119[FLOAT, 64]
  %onnx::Conv_121[FLOAT, 64x64x3x3]
  %onnx::Conv_122[FLOAT, 64]
  %onnx::Conv_124[FLOAT, 64x64x3x3]
  %onnx::Conv_125[FLOAT, 64]
  %onnx::Conv_127[FLOAT, 64x64x3x3]
  %onnx::Conv_128[FLOAT, 64]
  %onnx::Conv_130[FLOAT, 128x64x3x3]
  %onnx::Conv_131[FLOAT, 128]
  %onnx::Conv_133[FLOAT, 128x128x3x3]
  %onnx::Conv_134[FLOAT, 128]
  %onnx::Conv_136[FLOAT, 128x128x3x3]
  %onnx::Conv_137[FLOAT, 128]
  %onnx::Conv_139[FLOAT, 128x128x3x3]
  %onnx::Conv_140[FLOAT, 128]
) {
  %/block1/conv1/Conv_output_0 = Conv[dilations = [1, 1], group = 1, kernel_shape = [3, 3], pads = [1, 1, 1, 1], strides = [1, 

In [81]:
!pip install onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 82.1 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.4 MB/s eta 0:00:00


In [82]:


import onnxruntime as ort

session = ort.InferenceSession("/kaggle/working/resnet18_v1.onnx")
inputs = {session.get_inputs()[0].name: dummy_input.numpy()}
outputs = session.run(None, inputs)
print(outputs)

[array([[-0.02527974, -0.05944184,  0.04687503,  0.08315577,  0.00663027,
        -0.05959833, -0.03492719,  0.05199958,  0.06684899, -0.11986826]],
      dtype=float32)]


In [84]:

from torchvision import transforms
from PIL import Image
import numpy as np


image_path = "/kaggle/input/example-image/example1.jpg"  # Path to your 28x28 image
image = Image.open(image_path).convert("L")  # Convert to grayscale

# Define preprocessing steps
preprocess = transforms.Compose([
    transforms.Resize((28, 28)),  # Resize to 28x28 (if needed)
    transforms.ToTensor(),         # Convert to tensor
    transforms.Normalize((0.5,), (0.5,))  # Normalize (if required)
])

# Preprocess the image
input_tensor = preprocess(image)  # Shape: [1, 28, 28]
input_tensor = input_tensor.unsqueeze(0)  # Add batch dimension: [1, 1, 28, 28]

# Perform inference
with torch.no_grad():
    output = model(input_tensor)

# Interpret the output
predicted_class = torch.argmax(output, dim=1).item()
print(f"Predicted class: {predicted_class}")


Predicted class: 3
